In [28]:
#Read an Excel  file and extract 3 values,

import pandas as pd

# Path to the Excel file
file_path = r"C:\Users\Randall\Downloads\Listado de Movimiento de Bancos.xls"

# Read the Excel file
data = pd.read_excel(file_path)

# Extract the values from the specified columns
columns_to_extract = ['Recibo', 'Concepto', 'Débito']
extracted_data = data[columns_to_extract].copy()

# Split 'Concepto' at the first space into 'code' and 'name'
def split_concepto(value):
    if isinstance(value, str):
        parts = value.split(' ', 1)
        if len(parts) == 2:
            return pd.Series({'code': parts[0], 'name': parts[1]})
        else:
            return pd.Series({'code': parts[0], 'name': ''})
    else:
        return pd.Series({'code': '', 'name': ''})

extracted_data[['code', 'name']] = extracted_data['Concepto'].apply(split_concepto)

# Create 'flag' column: 1 if 'Recibo' and 'code' do not match, else 0
extracted_data['flag'] = (extracted_data['Recibo'].astype(str) != extracted_data['code'].astype(str)).astype(int)

# Create 'final' column: if 'Recibo' and 'code' are equal, use 'Recibo'; else use the longest one
extracted_data['final'] = extracted_data.apply(
    lambda row: row['Recibo'] if str(row['Recibo']) == str(row['code']) else max([str(row['Recibo']), str(row['code'])], key=len),
    axis=1
)

extracted_data['final']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


0     HMHCTBQDJ3
1     HM9F8N92TN
2     HMDXW9PN54
3     HMACKQQ9RA
4     HMFSKRMFRD
         ...    
66    HMFZMPW5RF
67    HMFQ224SCX
68    HMYXNBE4RW
69    HMYCTD22WH
70    HMM4BQKTAH
Name: final, Length: 71, dtype: object

In [29]:
import pandas as pd

# Path to the Excel file
file_path = r"C:\Users\Randall\Downloads\reservations (1).csv"

# Read the Excel file
data = pd.read_csv(file_path)
data['Ingresos'] = data['Ingresos'][1::]

Reservaciones = data[data['Estado'] == 'Huésped anterior']

# Display the first few rows of the dataframe

In [30]:
Reservaciones['Ingresos'].head()

0        NaN
1     $92.37
2    $103.46
3    $301.56
4    $510.00
Name: Ingresos, dtype: object

In [31]:
# Compare 'Recibo' in extracted_data with 'Código de confirmación' in Reservaciones
# Check for matches based on 'final' and 'Código de confirmación'
matches = extracted_data['final'].isin(Reservaciones['Código de confirmación'])

# Further check for matches by comparing 'Ingresos' and 'Débito'
matches_with_price_check = extracted_data[matches].merge(
    Reservaciones[['Código de confirmación', 'Ingresos']],
    left_on='final',
    right_on='Código de confirmación'
)
matches_with_price_check = matches_with_price_check[
    matches_with_price_check['Débito'].astype(float) == matches_with_price_check['Ingresos'].str.replace('$', '').str.replace(',', '').astype(float)
]

# Count the number of matches
num_matches = len(matches_with_price_check)
print(f"Number of matches with price check: {num_matches}")

# Check for non-matches
non_matches = extracted_data[~matches]

# Further check for non-matches by comparing 'Ingresos' and 'Débito'
non_matches_with_price_check = non_matches.merge(
    Reservaciones[['Código de confirmación', 'Ingresos']],
    left_on='final',
    right_on='Código de confirmación',
    how='inner'
)
non_matches_with_price_check = non_matches_with_price_check[
    non_matches_with_price_check['Débito'].astype(float) == non_matches_with_price_check['Ingresos'].str.replace('$', '').str.replace(',', '').astype(float)
]

# Print a message if there are non-matches with price check
if not non_matches_with_price_check.empty:
    print("There are non-matches that match by price:")
    print(non_matches_with_price_check)

Number of matches with price check: 38


In [32]:
# Get 'Código de confirmación' values from Reservaciones that did not match with extracted_data
non_matching_reservaciones = Reservaciones.loc[~Reservaciones['Código de confirmación'].isin(extracted_data['final']), 'Código de confirmación']
print(f"Number of non-matching 'Código de confirmación' values: {len(non_matching_reservaciones)}")
non_matching_reservaciones

Number of non-matching 'Código de confirmación' values: 0


Series([], Name: Código de confirmación, dtype: object)